## Preparing SV Dataset

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pylab as plt

#np.set_printoptions(formatter={'int':hex})

import os
files = os.listdir('sv_traces/')

### reads an sv_trace file into 2 structures:
### pcs: a list of 1001 16-bit pc values
### mem: a list of 1001 65542-byte memory vectors
def read_trace(sv_file):
    with open(sv_file, 'rb') as f:
        lines = f.read()
        lines = np.frombuffer(lines, dtype=np.uint8)
        lines = lines.reshape(1001,-1)
        pcs = lines[:, 0:2]
        mem = lines[:, :]
        #pcs = [np.uint16(i[1]<<8 | i[0]) for i in pcs]
        return mem, pcs
    
### reads trace data into a dataset of mem-to-pc mappings, where
### mem_current: partial memory trace &
### pcs_current: full pcs trace
def read_dataset(file):
    mem_trace = dict_traces[file][0]
    mem_current = mem_trace[:]
    pcs_trace = dict_traces[file][1]
    pcs_current = pcs_trace[:]
    return mem_current, pcs_current

### dict_traces[file][0]: memory trace of <file>
### dict_traces[file][1]: pc trace of <file>
dict_traces = {}
for file in files[0:20]:
    dict_traces[file] = read_trace(os.path.join('sv_traces/', file))
    
### dict_dataset[file][0]: memory trace of <file>
### dict_dataset[file][1]: pc trace of <file>
dict_dataset = {}
for trace in dict_traces:
    dict_dataset[trace] = read_dataset(trace)

In [2]:
dict_traces

{'167:149:9.svbinttrc': (array([[  0,   2,   0, ...,   2,   0,   0],
         [  1,   2,   0, ...,   2,   0,   0],
         [  3,   2,   0, ...,   2,   0,   0],
         ...,
         [  3,   2, 126, ...,   2,   0,   0],
         [  5,   2,  20, ...,   2,   0,   0],
         [  7,   2,  20, ...,   2,   0,   0]], dtype=uint8),
  array([[0, 2],
         [1, 2],
         [3, 2],
         ...,
         [3, 2],
         [5, 2],
         [7, 2]], dtype=uint8)),
 '121:187:13.svbinttrc': (array([[ 0,  2,  0, ...,  2,  0,  0],
         [ 1,  2,  0, ...,  2,  0,  0],
         [ 3,  2,  0, ...,  2,  0,  0],
         ...,
         [ 9,  2, 50, ...,  2,  0,  0],
         [10,  2, 50, ...,  2,  0,  0],
         [11,  2, 50, ...,  2,  0,  0]], dtype=uint8),
  array([[ 0,  2],
         [ 1,  2],
         [ 3,  2],
         ...,
         [ 9,  2],
         [10,  2],
         [11,  2]], dtype=uint8)),
 '144:101:13.svbinttrc': (array([[  0,   2,   0, ...,   2,   0,   0],
         [  1,   2,   0, ...,   2

In [3]:
dict_dataset

{'167:149:9.svbinttrc': (array([[  0,   2,   0, ...,   2,   0,   0],
         [  1,   2,   0, ...,   2,   0,   0],
         [  3,   2,   0, ...,   2,   0,   0],
         ...,
         [  3,   2, 126, ...,   2,   0,   0],
         [  5,   2,  20, ...,   2,   0,   0],
         [  7,   2,  20, ...,   2,   0,   0]], dtype=uint8),
  array([[0, 2],
         [1, 2],
         [3, 2],
         ...,
         [3, 2],
         [5, 2],
         [7, 2]], dtype=uint8)),
 '121:187:13.svbinttrc': (array([[ 0,  2,  0, ...,  2,  0,  0],
         [ 1,  2,  0, ...,  2,  0,  0],
         [ 3,  2,  0, ...,  2,  0,  0],
         ...,
         [ 9,  2, 50, ...,  2,  0,  0],
         [10,  2, 50, ...,  2,  0,  0],
         [11,  2, 50, ...,  2,  0,  0]], dtype=uint8),
  array([[ 0,  2],
         [ 1,  2],
         [ 3,  2],
         ...,
         [ 9,  2],
         [10,  2],
         [11,  2]], dtype=uint8)),
 '144:101:13.svbinttrc': (array([[  0,   2,   0, ...,   2,   0,   0],
         [  1,   2,   0, ...,   2

## Defining Neural Net

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

In [5]:
def format_label(l, width=16):
    val = bin(l)[2:]
    N = len(val)
    return [0.] * (width-N) + [float(i) for i in val]

In [6]:
format_label(512)

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [119]:
features, labels = dict_dataset[list(dict_dataset.keys())[0]]
for file in list(dict_dataset.keys())[1:10]:
    features = np.append(features, dict_dataset[file][0], axis=0)
    labels = np.append(labels, dict_dataset[file][1], axis=0)
features_test, labels_test = dict_dataset[list(dict_dataset.keys())[10]]
for file in list(dict_dataset.keys())[11:13]:
    features_test = np.append(features_test, dict_dataset[file][0], axis=0)
    labels_test = np.append(labels_test, dict_dataset[file][1], axis=0)

In [120]:
#features, labels = dict_dataset[list(dict_dataset.keys())[0]]
#features_test, labels_test = dict_dataset[list(dict_dataset.keys())[1]]
#features = features - features.mean(axis=0)
# temp
features_temp = features#[9:15, ]
labels_temp = labels#[9:15]
# end temp

In [9]:
print(labels_test[0:16])
print(labels_temp[0:16])


[[ 0  2]
 [ 1  2]
 [ 3  2]
 [ 5  2]
 [ 7  2]
 [ 8  2]
 [ 9  2]
 [10  2]
 [11  2]
 [12  2]
 [13  2]
 [14  2]
 [15  2]
 [16  2]
 [17  2]
 [18  2]]
[[ 0  2]
 [ 1  2]
 [ 3  2]
 [ 5  2]
 [ 7  2]
 [ 8  2]
 [ 9  2]
 [10  2]
 [11  2]
 [12  2]
 [13  2]
 [14  2]
 [15  2]
 [16  2]
 [ 3  2]
 [ 5  2]]


In [121]:
labels_temp[2]

array([3, 2], dtype=uint8)

In [122]:
partial_mem_trace = features_temp[: , 0:8]
#partial_mem_trace = features_temp[: , 0:1024]
features_temp = partial_mem_trace # all rows are identical here

partial_mem_trace_test = features_test[: , 0:8]
features_test = partial_mem_trace_test # all rows are identical here

    
print(partial_mem_trace.shape[0])
print(features_temp.shape[0])
print(partial_mem_trace.shape[1])
print(features_temp.shape[1])

10010
10010
8
8


In [123]:
features_temp[0]

array([ 0,  2,  0,  0,  0,  0, 20,  0], dtype=uint8)

### Training NN

In [124]:
net = nn.Sequential(
    nn.Linear(features_temp.shape[1], 2**8),
    nn.ReLU(), #rectified linear unit
    #nn.Linear(2**12, 2**10),
    #nn.ReLU(),
    #nn.Linear(2**8, 2**8),
    #nn.ReLU(),
    nn.Linear(2**8, 2),
    #nn.Sigmoid()
)

#criterion = nn.BCELoss()
#criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()

def train(n_epochs, net, features, labels, criterion, optimizer, freq=1, debug=False):
    for i in range(n_epochs): #number of passes over full dataset
        #step 1: get features and use net to make predictions
        preds = net(torch.from_numpy(features).float())
        
        #step 2: compute loss/error
        labels_torch = torch.tensor(torch.from_numpy(np.array(labels)).float())
        if debug: print('\n------')
        if debug: print(preds)
        #loss = np.sqrt(((np.rint(preds.detach().numpy()) - labels_torch.detach().numpy())**2).mean())
        loss = criterion(preds, labels_torch)
        if i % freq == 0:
            print('epoch:', i, 'loss:', loss)

        #step 3: backprop to update weights
        # compute gradients/derivatives - backprop
        # use gradients to update weights - gradient descent - w = w - 0.1 * deriv. loss w.r.t. w

        optimizer.zero_grad() #set previous buffers to zero
        loss.backward() #backprop
        optimizer.step() #update weights        
        
    return net

optimizer = optim.Adam(net.parameters(), lr=1e-5)
print(features_temp)
#net = train(1000, net, features_temp, labels_temp, criterion, optimizer, debug=False)

[[  0   2   0 ...   0  20   0]
 [  1   2   0 ...   0  20   0]
 [  3   2   0 ...   0  22   0]
 ...
 [  5   2 145 ...   0 212   0]
 [  7   2 145 ...   0 212   0]
 [  8   2 145 ...   0 212   0]]


In [131]:
net = train(10000, net, features_temp, labels_temp, criterion, optimizer, debug=False)

/tmp/ipykernel_39771/1586916469.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_torch = torch.tensor(torch.from_numpy(np.array(labels)).float())


epoch: 0 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 1 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 2 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 3 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 4 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 5 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 6 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 7 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 8 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 9 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 10 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 11 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 12 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 13 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 14 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 15 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 16 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 

epoch: 147 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 148 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 149 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 150 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 151 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 152 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 153 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 154 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 155 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 156 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 157 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 158 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 159 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 160 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 161 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 162 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 163 loss: tensor(0.0001, grad_fn=

epoch: 296 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 297 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 298 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 299 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 300 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 301 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 302 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 303 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 304 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 305 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 306 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 307 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 308 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 309 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 310 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 311 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 312 loss: tensor(0.0001, grad_fn=

epoch: 445 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 446 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 447 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 448 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 449 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 450 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 451 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 452 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 453 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 454 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 455 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 456 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 457 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 458 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 459 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 460 loss: tensor(0.0001, grad_fn=<MseLossBackward0>)
epoch: 461 loss: tensor(0.0001, grad_fn=

...

epoch: 9304 loss: tensor(8.3729e-05, grad_fn=<MseLossBackward0>)
epoch: 9305 loss: tensor(8.3573e-05, grad_fn=<MseLossBackward0>)
epoch: 9306 loss: tensor(8.3463e-05, grad_fn=<MseLossBackward0>)
epoch: 9307 loss: tensor(8.3385e-05, grad_fn=<MseLossBackward0>)
epoch: 9308 loss: tensor(8.3327e-05, grad_fn=<MseLossBackward0>)
epoch: 9309 loss: tensor(8.3288e-05, grad_fn=<MseLossBackward0>)
epoch: 9310 loss: tensor(8.3269e-05, grad_fn=<MseLossBackward0>)
epoch: 9311 loss: tensor(8.3272e-05, grad_fn=<MseLossBackward0>)
epoch: 9312 loss: tensor(8.3298e-05, grad_fn=<MseLossBackward0>)
epoch: 9313 loss: tensor(8.3347e-05, grad_fn=<MseLossBackward0>)
epoch: 9314 loss: tensor(8.3410e-05, grad_fn=<MseLossBackward0>)
epoch: 9315 loss: tensor(8.3483e-05, grad_fn=<MseLossBackward0>)
epoch: 9316 loss: tensor(8.3555e-05, grad_fn=<MseLossBackward0>)
epoch: 9317 loss: tensor(8.3618e-05, grad_fn=<MseLossBackward0>)
epoch: 9318 loss: tensor(8.3665e-05, grad_fn=<MseLossBackward0>)
epoch: 9319 loss: tensor(

epoch: 9450 loss: tensor(8.3190e-05, grad_fn=<MseLossBackward0>)
epoch: 9451 loss: tensor(8.3125e-05, grad_fn=<MseLossBackward0>)
epoch: 9452 loss: tensor(8.3073e-05, grad_fn=<MseLossBackward0>)
epoch: 9453 loss: tensor(8.3059e-05, grad_fn=<MseLossBackward0>)
epoch: 9454 loss: tensor(8.3098e-05, grad_fn=<MseLossBackward0>)
epoch: 9455 loss: tensor(8.3200e-05, grad_fn=<MseLossBackward0>)
epoch: 9456 loss: tensor(8.3365e-05, grad_fn=<MseLossBackward0>)
epoch: 9457 loss: tensor(8.3598e-05, grad_fn=<MseLossBackward0>)
epoch: 9458 loss: tensor(8.3901e-05, grad_fn=<MseLossBackward0>)
epoch: 9459 loss: tensor(8.4292e-05, grad_fn=<MseLossBackward0>)
epoch: 9460 loss: tensor(8.4804e-05, grad_fn=<MseLossBackward0>)
epoch: 9461 loss: tensor(8.5480e-05, grad_fn=<MseLossBackward0>)
epoch: 9462 loss: tensor(8.6373e-05, grad_fn=<MseLossBackward0>)
epoch: 9463 loss: tensor(8.7520e-05, grad_fn=<MseLossBackward0>)
epoch: 9464 loss: tensor(8.8935e-05, grad_fn=<MseLossBackward0>)
epoch: 9465 loss: tensor(

epoch: 9593 loss: tensor(8.2484e-05, grad_fn=<MseLossBackward0>)
epoch: 9594 loss: tensor(8.2491e-05, grad_fn=<MseLossBackward0>)
epoch: 9595 loss: tensor(8.2509e-05, grad_fn=<MseLossBackward0>)
epoch: 9596 loss: tensor(8.2526e-05, grad_fn=<MseLossBackward0>)
epoch: 9597 loss: tensor(8.2537e-05, grad_fn=<MseLossBackward0>)
epoch: 9598 loss: tensor(8.2540e-05, grad_fn=<MseLossBackward0>)
epoch: 9599 loss: tensor(8.2539e-05, grad_fn=<MseLossBackward0>)
epoch: 9600 loss: tensor(8.2542e-05, grad_fn=<MseLossBackward0>)
epoch: 9601 loss: tensor(8.2555e-05, grad_fn=<MseLossBackward0>)
epoch: 9602 loss: tensor(8.2585e-05, grad_fn=<MseLossBackward0>)
epoch: 9603 loss: tensor(8.2643e-05, grad_fn=<MseLossBackward0>)
epoch: 9604 loss: tensor(8.2730e-05, grad_fn=<MseLossBackward0>)
epoch: 9605 loss: tensor(8.2866e-05, grad_fn=<MseLossBackward0>)
epoch: 9606 loss: tensor(8.3065e-05, grad_fn=<MseLossBackward0>)
epoch: 9607 loss: tensor(8.3363e-05, grad_fn=<MseLossBackward0>)
epoch: 9608 loss: tensor(

epoch: 9741 loss: tensor(8.2794e-05, grad_fn=<MseLossBackward0>)
epoch: 9742 loss: tensor(8.2603e-05, grad_fn=<MseLossBackward0>)
epoch: 9743 loss: tensor(8.2338e-05, grad_fn=<MseLossBackward0>)
epoch: 9744 loss: tensor(8.2157e-05, grad_fn=<MseLossBackward0>)
epoch: 9745 loss: tensor(8.2143e-05, grad_fn=<MseLossBackward0>)
epoch: 9746 loss: tensor(8.2257e-05, grad_fn=<MseLossBackward0>)
epoch: 9747 loss: tensor(8.2389e-05, grad_fn=<MseLossBackward0>)
epoch: 9748 loss: tensor(8.2446e-05, grad_fn=<MseLossBackward0>)
epoch: 9749 loss: tensor(8.2393e-05, grad_fn=<MseLossBackward0>)
epoch: 9750 loss: tensor(8.2271e-05, grad_fn=<MseLossBackward0>)
epoch: 9751 loss: tensor(8.2155e-05, grad_fn=<MseLossBackward0>)
epoch: 9752 loss: tensor(8.2099e-05, grad_fn=<MseLossBackward0>)
epoch: 9753 loss: tensor(8.2119e-05, grad_fn=<MseLossBackward0>)
epoch: 9754 loss: tensor(8.2178e-05, grad_fn=<MseLossBackward0>)
epoch: 9755 loss: tensor(8.2230e-05, grad_fn=<MseLossBackward0>)
epoch: 9756 loss: tensor(

epoch: 9887 loss: tensor(9.0447e-05, grad_fn=<MseLossBackward0>)
epoch: 9888 loss: tensor(8.9100e-05, grad_fn=<MseLossBackward0>)
epoch: 9889 loss: tensor(8.7061e-05, grad_fn=<MseLossBackward0>)
epoch: 9890 loss: tensor(8.4822e-05, grad_fn=<MseLossBackward0>)
epoch: 9891 loss: tensor(8.2985e-05, grad_fn=<MseLossBackward0>)
epoch: 9892 loss: tensor(8.1975e-05, grad_fn=<MseLossBackward0>)
epoch: 9893 loss: tensor(8.1885e-05, grad_fn=<MseLossBackward0>)
epoch: 9894 loss: tensor(8.2483e-05, grad_fn=<MseLossBackward0>)
epoch: 9895 loss: tensor(8.3366e-05, grad_fn=<MseLossBackward0>)
epoch: 9896 loss: tensor(8.4128e-05, grad_fn=<MseLossBackward0>)
epoch: 9897 loss: tensor(8.4477e-05, grad_fn=<MseLossBackward0>)
epoch: 9898 loss: tensor(8.4313e-05, grad_fn=<MseLossBackward0>)
epoch: 9899 loss: tensor(8.3734e-05, grad_fn=<MseLossBackward0>)
epoch: 9900 loss: tensor(8.2979e-05, grad_fn=<MseLossBackward0>)
epoch: 9901 loss: tensor(8.2316e-05, grad_fn=<MseLossBackward0>)
epoch: 9902 loss: tensor(

In [148]:
#net = net.to(device)
N_epochs = 10000

labels_temp = np.array(labels_temp)
#train_data_torch = torch.from_numpy(features_temp).float()
#train_target_torch = torch.from_numpy(labels_temp).float()
train_target_torch = torch.tensor(torch.from_numpy(np.array(labels_temp)).float())

import sys
np.set_printoptions(threshold=sys.maxsize)

def validate(net, test_data, test_target, i):
    
    test_data_torch = torch.from_numpy(features_test).float()
    #test_target_torch = torch.from_numpy(labels_test).float()
    test_target_torch = torch.tensor(torch.from_numpy(np.array(test_target)).float())

    test_pred = np.rint(net(test_data_torch).detach().numpy())
    if i % 500 == 0:
        print(np.concatenate((test_pred, test_target_torch.detach().numpy()), axis=1))
    #test_accuracy = (test_target_torch.detach().numpy() == test_pred).float().mean().detach().numpy()
    test_accuracy = (test_target_torch.detach().numpy() == test_pred).mean()

    #test_rmse = ((test_pred-test_target_torch)**2).mean().sqrt().detach().numpy()
    test_rmse = (np.sqrt((test_pred-test_target_torch.detach().numpy())**2).mean())

    return test_accuracy, test_rmse

for i in range(1):
    #pred = net(train_data_torch)
    #loss = criterion(pred, train_target_torch)
    
    #optimizer.zero_grad()
    #loss.backward()
    #optimizer.step()
    
    if i % 100 == 0:
        #train_accuracy, train_rmse = validate(net, features_temp, labels_temp)
        test_accuracy, test_rmse = validate(net, features_test, labels_test, i)
        #print(f'Epoch = {i} loss={loss} train_acc = {train_accuracy:.3f} test_acc = {test_accuracy:.3f} train_rmse = {train_rmse:.3f}  test_rmse = {test_rmse:.3f}')         
        print(f'Epoch = {i} test_acc = {test_accuracy:.3f}  test_rmse = {test_rmse:.3f}') 

[[ 0.  2.  0.  2.]
 [ 1.  2.  1.  2.]
 [ 3.  2.  3.  2.]
 [ 5.  2.  5.  2.]
 [ 8.  2.  8.  2.]
 [ 9.  2.  9.  2.]
 [10.  2. 10.  2.]
 [11.  2. 11.  2.]
 [12.  2. 12.  2.]
 [13.  2. 13.  2.]
 [14.  2. 14.  2.]
 [15.  2. 15.  2.]
 [16.  2. 16.  2.]
 [17.  2. 17.  2.]
 [ 3.  2.  3.  2.]
 [ 5.  2.  5.  2.]
 [ 8.  2.  8.  2.]
 [ 9.  2.  9.  2.]
 [10.  2. 10.  2.]
 [11.  2. 11.  2.]
 [12.  2. 12.  2.]
 [13.  2. 13.  2.]
 [14.  2. 14.  2.]
 [15.  2. 15.  2.]
 [16.  2. 16.  2.]
 [17.  2. 17.  2.]
 [ 3.  2.  3.  2.]
 [ 5.  2.  5.  2.]
 [ 8.  2.  8.  2.]
 [ 9.  2.  9.  2.]
 [10.  2. 10.  2.]
 [11.  2. 11.  2.]
 [12.  2. 12.  2.]
 [13.  2. 13.  2.]
 [14.  2. 14.  2.]
 [15.  2. 15.  2.]
 [16.  2. 16.  2.]
 [17.  2. 17.  2.]
 [ 3.  2.  3.  2.]
 [ 5.  2.  5.  2.]
 [ 8.  2.  8.  2.]
 [ 9.  2.  9.  2.]
 [10.  2. 10.  2.]
 [11.  2. 11.  2.]
 [12.  2. 12.  2.]
 [13.  2. 13.  2.]
 [14.  2. 14.  2.]
 [15.  2. 15.  2.]
 [16.  2. 16.  2.]
 [17.  2. 17.  2.]
 [ 3.  2.  3.  2.]
 [ 5.  2.  5.  2.]
 [ 8.  2.  8

/tmp/ipykernel_39771/3030192740.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_target_torch = torch.tensor(torch.from_numpy(np.array(labels_temp)).float())
/tmp/ipykernel_39771/3030192740.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_target_torch = torch.tensor(torch.from_numpy(np.array(test_target)).float())


In [134]:
preds = net(torch.from_numpy(features_test[3]).float()).detach().numpy()
print(np.rint(preds))

[5. 2.]


In [67]:
for p in preds:
    print(p)

tensor(4.9992, grad_fn=<UnbindBackward0>)
tensor(2.0004, grad_fn=<UnbindBackward0>)


In [60]:
labels_test[3]

array([5, 2], dtype=uint8)